## Kalasim Spark Demo

In this notebook, we'll demonstrate how to parallelize simulation workload with kalasim.

In [1]:
%useLatestDescriptors

In [2]:
%use spark(v=1.2.2, spark=3.3.1, scala=2.12, spark.master=local[4], spark.app.name="kalasim-demo", displayLimit=30, displayTruncate=20)

received properties: Properties: {v=1.2.2, spark=3.3.1, scala=2.12, spark.master=local[4], spark.app.name=kalasim-demo, displayLimit=30, displayTruncate=20, spark.sql.codegen.wholeStage=false, fs.hdfs.impl=org.apache.hadoop.hdfs.DistributedFileSystem, fs.file.impl=org.apache.hadoop.fs.LocalFileSystem}, providing Spark with: {spark.master=local[4], spark.app.name=kalasim-demo, spark.sql.codegen.wholeStage=false, fs.hdfs.impl=org.apache.hadoop.hdfs.DistributedFileSystem, fs.file.impl=org.apache.hadoop.fs.LocalFileSystem}
Spark session (Spark: 3.3.1, Scala: 2.12, v: 1.2.2)  has been started and is running. No `withSpark { }` necessary, you can access `spark` and `sc` directly. To use Spark streaming, use `%use spark-streaming` instead.


In [3]:
@file:Repository("*mavenLocal")

// @file:DependsOn("com.github.holgerbrandl:kalasim:0.7-SNAPSHOT")
//@file:DependsOn("io.insert-koin:koin-core:3.2.2")
@file:DependsOn("com.github.holgerbrandl:kalasim:0.8")
//@file:DependsOn("com.github.holgerbrandl:kravis:0.8.5")

println("loaded dependencies")

[ERROR] Failed to load library integration class 'org.kalasim.analysis.NotebookIntegration'
org/koin/core/component/KoinComponent
java.lang.NoClassDefFoundError: org/koin/core/component/KoinComponent
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1012)


loaded dependencies


	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/java.net.URLClassLoader.defineClass(URLClassLoader.java:524)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:427)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:421)
	at java.base/java.security.AccessController.doPrivileged(AccessController.java:712)
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:420)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:587)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:520)
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1012)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/java.net.URLClassLoader.defineClass(URLClassLoader.java:524)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:427)
	at java.base/java.net.URLClassLoader$1.ru

In [4]:
import org.kalasim.TickTransform
import kotlin.time.DurationUnit
import kotlin.time.Duration.Companion.seconds

Build sim model

In [5]:
data class SimConfig(val run: Int)

val configs = List(10) { SimConfig(it) }

//https://www.learningjournal.guru/article/apache-spark/apache-spark-parallel-processing/

val dd = configs.toDS().map { it ->
//            Math.random()
    org.kalasim.Environment().apply {
        tickTransform = TickTransform(DurationUnit.DAYS)
        run(1.seconds)
    }.currentComponent?.componentState?.name
}

In [6]:

dd.show()



Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (NTNB608.dd.systemagmbh.de executor driver): java.lang.NoClassDefFoundError: org/koin/core/component/KoinComponent
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1012)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/java.net.URLClassLoader.defineClass(URLClassLoader.java:524)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:427)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:421)
	at java.base/java.security.AccessController.doPrivileged(AccessController.java:712)
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:420)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:587)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:520)
	at java.base/java.lang.ClassLo

Explore parallelization of sim runs

In [ ]:

dd.rdd().numPartitions

// repartition data set
//https://sparkbyexamples.com/spark/spark-partitioning-understanding/

dd.repartition(3)

dd.rdd().numPartitions
